In [1]:
import os.path
import heapq
from geopandas import GeoDataFrame, GeoSeries, read_file
from geopandas.tools import sjoin
from shapely.geometry import Point, Polygon
import scipy.spatial as spatial
from pandas import read_csv, to_numeric
from math import log
from numpy import where, concatenate, mean

import jupynbimp
import restaurants_data_cleaning, review_data_process, census2010_data_cleaning, acs2013_data_cleaning

importing Jupyter notebook from restaurants_data_cleaning.ipynb
importing Jupyter notebook from review_data_process.ipynb
importing Jupyter notebook from review_data_getData.ipynb
importing Jupyter notebook from review_data_classify.ipynb
importing Jupyter notebook from census2010_data_cleaning.ipynb
importing Jupyter notebook from acs2013_data_cleaning.ipynb


In [4]:
# Source CRS: WGS 84/Lat,Long; Projected CRS: WGS 84/UTM Zone 12N
CRS = {'GCS':'+init=epsg:4326', 'projected':'+init=epsg:32612'}
USE_CACHE=True # if false, forces all modules to re-compute variables (takes significantly more time!) 
SQ_METERS_PER_SQ_KM=1000000

In [3]:
def normalizeByArea(geoDataFrame, column):
    return geoDataFrame.apply(
        lambda row: row[column]/row['area_sqkm'], axis=1
    ) 

## Transform data into consolidated shapefile

In [7]:
def getData(fromCache=True):
    # Retrieved 09-07-2016 from https://nz.yelp.com/dataset_challenge/dataset 
    dataDirectory = '../../data/'
    outputName = 'restaurants_merge_clean.csv'
    
    if fromCache & os.path.isfile(dataDirectory + outputName ):
        return read_csv(dataDirectory + outputName, header=0)
    
    else:
        
        # merge Yelp data
        reviews = (review_data_process.getData(fromCache=fromCache)
                   .set_index('business_id')
                  )
        
        restaurants = (restaurants_data_cleaning.getData(fromCache=fromCache)
                       .set_index('business_id')
                       .join(reviews, how='inner', 
                             lsuffix='business_id', 
                             rsuffix='business_id')
                       .rename(columns={'business_idbusiness_id':'business_id'})
                      )
        
        # create spatial features
        restaurants = GeoDataFrame(
            restaurants,
            geometry=[Point(x,y) 
                      for x, y in zip(restaurants['longitude'],
                                      restaurants['latitude'])
                     ],
            crs=CRS['GCS']
        )
        
        restaurants.to_crs(CRS['projected'], inplace=True)
        restaurants.drop(['latitude', 'longitude'], axis=1, inplace=True)
        
        # Phoenix Convention Center in projected coordinates
        phoenixCBD = GeoSeries(Point(400557, 3701492), crs=CRS['projected'])

        restaurants['dist_CBD'] = restaurants.geometry.apply(
            lambda point: phoenixCBD.distance(point)
        )
        
        # Scottsdale shopping district in projected coordinates
        scottsdale = GeoSeries(Point(413975, 3707095), crs=CRS['projected'])
        
        restaurants['dist_scottsdale'] = restaurants.geometry.apply(
            lambda point: scottsdale.distance(point)
        )
        
        # calculate distance from freeway exits
        if fromCache & os.path.isfile(dataDirectory + 'shapefiles/motorway-exits/motorwayExits.shp'):
            motorwayExits = read_file(dataDirectory + 'shapefiles/motorway-exits/motorwayExits.shp')
            
        else:
            
            # Retrieved 17/07/2016 from http://download.geofabrik.de/north-america/us/arizona.html
            motorwayExits = read_file('/arizona-latest/roads.shp', 
                                     vfs='zip://' + dataDirectory + 'shapefiles/arizona-latest.zip'
                                    ).to_crs(CRS['projected'])
            
            motorwayExits = (motorwayExits[motorwayExits['type'] == 'motorway_link'])
            
            # cache as shapefile
            motorwayExits.to_file(dataDirectory + 'shapefiles/motorway-exits/motorwayExits.shp')
            
            
        restaurants['dist_mwy_exit'] = restaurants.geometry.apply(
           lambda point: motorwayExits.distance(point).min()
        )
        
        
        # for efficient nearest-neighbor queries
        c2DTree = spatial.cKDTree(concatenate(restaurants.geometry.apply(lambda point: list(point.coords))))
    
        restaurants['nearest_neighbor_distance'] = restaurants.geometry.apply(
           lambda point: c2DTree.query(list(point.coords), k=[2])[0][0][0]
        )
        
        # competitor proximity = R statistic = r_obs / r_exp:
        # r_obs = mean nearest neighbor distance of 5 nearest neighbors 
        # r_exp = 1/(2(n/study area)^0.5)(Lu & Wong, 2008)
        r_exp = 1 / (2 * (restaurants.shape[0] / restaurants.geometry.unary_union.envelope.area)**0.5)
        restaurants['competitor_proximity'] = restaurants.geometry.apply(
           lambda point: (restaurants.iloc
                          [c2DTree.query(list(point.coords), k=[2, 3, 4, 5, 6])[1][0]]
                          ['nearest_neighbor_distance']
                          .mean() / r_exp
                         )
        ) 
        
        # merge demographic (US census) data
        census2010 = census2010_data_cleaning.getData(fromCache=USE_CACHE)

        # Retrieved 10-07-2016 from https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2010&layergroup=Block+Groups
        blockGroups2010 = read_file('/tl_2010_04_bg10.shp', 
                                    vfs='zip://' + dataDirectory + 'shapefiles/tl_2010_04_bg10.zip')
        blockGroups2010.GEOID10 = to_numeric(blockGroups2010.GEOID10)
        blockGroups2010 = (blockGroups2010[['GEOID10','geometry']]
                           .rename(columns={'GEOID10':'GEOID'})
                           .to_crs(CRS['projected'])
                          )
        blockGroups2010['area_sqkm'] = blockGroups2010.geometry.area/SQ_METERS_PER_SQ_KM 
        blockGroups2010 = blockGroups2010.merge(census2010, on='GEOID')

        # Normalize count data by block group area
        blockGroups2010['population_density'] = normalizeByArea(blockGroups2010,'population_total')
        blockGroups2010['home_mortgage_density'] = normalizeByArea(blockGroups2010,'home_mortgages')
        blockGroups2010['home_owner_density'] = normalizeByArea(blockGroups2010,'home_owners')
        blockGroups2010['renter_density'] = normalizeByArea(blockGroups2010,'renters')
        blockGroups2010['household_density'] = normalizeByArea(blockGroups2010,'total_households')
        blockGroups2010['family_household_density'] = normalizeByArea(blockGroups2010,'family_households')
        blockGroups2010['single_household_density'] = normalizeByArea(blockGroups2010,'single_households')
        blockGroups2010['hispanic_latino_population_density'] = normalizeByArea(blockGroups2010,'population_hispanic_latino')
        blockGroups2010['white_population_density'] = normalizeByArea(blockGroups2010,'population_white')
        blockGroups2010['black_population_density'] = normalizeByArea(blockGroups2010,'population_black')
        blockGroups2010['native_american_population_density'] = normalizeByArea(blockGroups2010,'population_native_american')
        blockGroups2010['asian_population_density'] = normalizeByArea(blockGroups2010,'population_asian')

        blockGroups2010.drop(['population_total',
                              'home_mortgages',
                              'home_owners',
                              'renters',
                              'total_households',
                              'family_households',
                              'single_households','population_hispanic_latino',
                              'population_white',
                              'population_black',
                              'population_native_american',
                              'population_asian'
                             ], 
                             axis=1, inplace=True
                            )
        
        acs2013 = acs2013_data_cleaning.getData(fromCache=False) 

        # Retrieved 10-07-2016 https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2013&layergroup=Block+Groups
        blockGroups2013 = read_file('/tl_2013_04_bg.shp', 
                                    vfs='zip://' + dataDirectory + 'shapefiles/tl_2013_04_bg.zip')
        blockGroups2013.GEOID = to_numeric(blockGroups2010.GEOID)
        blockGroups2013 = (blockGroups2013[['GEOID','geometry']]
                           .to_crs(CRS['projected'])
                          )

        blockGroups2013['area_sqkm'] = blockGroups2013.geometry.area/SQ_METERS_PER_SQ_KM 
        blockGroups2013 = blockGroups2013.merge(acs2013, on='GEOID')

        # Normalize count data by block group area
        blockGroups2013['density_education_highschool'] = normalizeByArea(blockGroups2013,'education_highschool')
        blockGroups2013['density_education_undergraduate'] = normalizeByArea(blockGroups2013,'education_undergraduate')
        blockGroups2013['density_education_postgraduate'] = normalizeByArea(blockGroups2013,'education_postgraduate')

        blockGroups2013.drop(['education_highschool',
                              'education_undergraduate',
                              'education_postgraduate'
                             ], 
                             axis=1, inplace=True
                            )
        
        # spatial join restaurants with census block groups
        restaurants.reset_index(inplace=True)
        restaurants_census2010 = (sjoin(restaurants[['business_id', 'geometry']], blockGroups2010, how='inner')
                                  .drop(['geometry', 'index_right', 'GEOID', 'area_sqkm'], axis=1)
                                 )
        restaurants_acs2013 = (sjoin(restaurants[['business_id', 'geometry']], blockGroups2013, how='inner')
                               .drop(['geometry', 'index_right', 'GEOID', 'area_sqkm'], axis=1)
                              )
        restaurants = GeoDataFrame(restaurants_census2010.merge(restaurants_acs2013)
                                   .merge(restaurants), 
                                   crs=CRS['projected']
                                  )
  
        return restaurants